# Data Collection

In order to begin analyzing stocks we first need to collect data to analyze. We will do this by scraping a specified timeframe of stock information for a specified number of stocks.


## Getting Started
To get started, we must import some libraries:

In [1]:
import sys
!{sys.executable} -m pip install yfinance
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
print("======================")


import yfinance as yf
import pandas as pd

print ("Successfully imported libraries")

Successfully imported libraries


We will begin by specifying a list of tickers we want to scrape:

In [2]:
tickers = ["AMZN", "TSLA", "BABA", "HD", "TM", "NKE", "MCD", "LOW", "SBUX", "JD", "PDD", "BKNG", "GM", "MELI", "TJX", "NIO", "F", "LULU", "HMC", "CMG"]

From here, we have to define some options:

- Interval: This represents the frequency we want to get stock data
- Time Frame (2 options):
    - A)
        - Period: This represents a timeframe in english (1m, 5m, 1d, 1y, ...)
    - B)
        - Start Date: This is the starting date for data scraping
        - End Date: This is the ending date for data scraping

In [3]:
period = '10y'
start_date = None
end_date = None
interval = "1d"

Just a quick check to ensure variables are instantiated correctly:

In [4]:
if tickers == None:
    raise Exception("You must specify a list of tickers to scrape")

if period == None and start_date == None and end_date == None:
            raise Exception("You must specify one timeframe in order to scrape")
        
if period == None and (start_date == None or end_date == None):
    raise Exception("You must specify both ends of the timeframe in order to scrape")

if period not in ['1d','5d','1mo','3mo','6mo','1y','2y','5y','10y','ytd','max']:
    raise Exception("Please input a valid period")

if interval not in ['1m','2m','5m','15m','30m','60m','90m','1h','1d','5d','1wk','1mo','3mo']:
    raise Exception("Please input a valid time interval")

if period != None and (start_date != None or end_date != None):
    raise Exception("You can only specify one type of timeframe in order to scrape")

print("Passed!")

Passed!


## Scraping Data
Now that we have defined the scraping parameters, we can actually begin to scrape. We do this by calling the YFinance download method for each stock. However, we have to consider two cases:
- We are using period
- We are using start/end dates


In [5]:
ticker_string = ' '.join(tickers) #convert from list to space-separated string
print(f"Ticker String: \n{ticker_string}")

if period != None: #if using period
    data = yf.download(
        ticker_string,
        period = period,
        interval = interval,
        group_by = 'ticker',
        threads = True
    )
else:
    data = yf.download( #if using start/end dates
        ticker_string,
        start = start_date, 
        end = end_date,
        interval = interval,
        group_by = 'ticker',
        threads = True
    )


data = data.drop([(i, 'Close') for i in tickers], axis=1)
data = data.rename({"Adj Close": "Close"}, axis=1)


data.head()

Ticker String: 
AMZN TSLA BABA HD TM NKE MCD LOW SBUX JD PDD BKNG GM MELI TJX NIO F LULU HMC CMG
[*********************100%***********************]  20 of 20 completed


NIO                               CMG                          \
           Open High Low Close Volume        Open        High         Low   
Date                                                                        
2011-11-14  NaN  NaN NaN   NaN    NaN  330.750000  334.339996  326.329987   
2011-11-15  NaN  NaN NaN   NaN    NaN  326.000000  329.109985  322.510010   
2011-11-16  NaN  NaN NaN   NaN    NaN  325.029999  326.950012  315.010010   
2011-11-17  NaN  NaN NaN   NaN    NaN  316.709991  319.640015  310.000000   
2011-11-18  NaN  NaN NaN   NaN    NaN  314.299988  315.899994  308.260010   

                                ...        MCD                        \
                 Close  Volume  ...       Open       High        Low   
Date                            ...                                    
2011-11-14  327.029999  464600  ...  94.349998  94.800003  93.860001   
2011-11-15  327.459991  496600  ...  93.870003  94.910004  93.699997   
2011-11-16  316.670013  791900  ...  93.910004  94.330002  93.070000   
2011-11-17  313.609985  710200  ...  93.139999  93.550003  91.720001   
2011-11-18  311.059998  506100  ...  92.559998  93.000000  92.190002   

                                    TJX                                         
                Close   Volume     Open     High      Low      Close    Volume  
Date                                                                            
2011-11-14  70.703171  3232200  15.2300  15.4275  15.1150  13.360164  10450800  
2011-11-15  71.011345  5577300  15.0675  15.3575  14.8675  13.426335  13080000  
2011-11-16  70.019150  3915600  15.1275  15.3800  15.0775  13.324871  13473200  
2011-11-17  69.372665  5435000  15.0375  15.1250  14.7125  13.053565  10610000  
2011-11-18  69.710953  4791500  14.8200  14.9725  14.7625  13.132973  13084000  

[5 rows x 100 columns]

# Adding Technical Indicators

Using the [TA-Lib](https://mrjbq7.github.io/ta-lib/) package, we can broadcast technical indicators across the time series into their own dataframe columns

In [7]:
import talib
!{sys.executable} -m pip install TA-Lib

def computeRSI(df):
    return talib.RSI(df["Close"], timeperiod=14)

def computeUltimateOscillator(df):
    return talib.ULTOSC(df["High"], df["Low"], df["Close"], timeperiod1=7, timeperiod2=14, timeperiod3=28)

def computeBollingerBands(df):
    upperband, middleband, _ = talib.BBANDS(df["Close"], timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
    return (upperband - middleband) / df["Close"] # need to normalize so value is not dependent on share price

def computeChaikinOscillator(df):
    return talib.ADOSC(df["High"], df["Low"], df["Close"], df["Volume"], fastperiod=3, slowperiod=10)

def computeNATR(df):
    return talib.NATR(df["High"], df["Low"], df["Close"], timeperiod=14)

def computeSMA(df, window):
    return talib.SMA(df["Close"], timeperiod=window) / df["Close"] # need to normalize so value is not dependent on share price

def computeSAR(df):
    return talib.SAR(df["High"], df["Low"], acceleration=0, maximum=0) / df["Close"] # need to normalize so value is not dependent on share price

def computeWilliamsR(df):
    return talib.WILLR(df["High"], df["Low"], df["Close"], timeperiod=14)

def computeAPO(df):
    return talib.APO(df["Close"], fastperiod=12, slowperiod=26, matype=0) / df["Close"]# need to normalize so value is not dependent on share price

def computeROC(df, window):
    return talib.ROCP(df["Close"], timeperiod=window) 


Now, we can apply these functions across the collected timeseries

In [8]:
for ticker in tickers:
    df = data[ticker]
    data[ticker, 'RSI'] = computeRSI(df)
    data[ticker, 'Ultimate'] = computeUltimateOscillator(df)
    data[ticker, 'BandRadius'] = computeBollingerBands(df)
    data[ticker, 'Chaikin'] = computeChaikinOscillator(df)
    data[ticker, 'NATR'] = computeNATR(df)
    data[ticker, 'SMA_5'] = computeSMA(df, 5)
    data[ticker, 'SMA_20'] = computeSMA(df, 20)
    data[ticker, 'SMA_100'] = computeSMA(df, 100)
    data[ticker, 'SAR'] = computeSAR(df)
    data[ticker, 'Williams%R'] = computeWilliamsR(df)
    data[ticker, 'APO'] = computeAPO(df)
    data[ticker, 'ROC_5'] = computeROC(df, 5)
    data[ticker, 'ROC_20'] = computeROC(df, 20)
    data[ticker, 'ROC_100'] = computeROC(df, 100)

data.head(100)

NIO                               CMG                          \
           Open High Low Close Volume        Open        High         Low   
Date                                                                        
2011-11-14  NaN  NaN NaN   NaN    NaN  330.750000  334.339996  326.329987   
2011-11-15  NaN  NaN NaN   NaN    NaN  326.000000  329.109985  322.510010   
2011-11-16  NaN  NaN NaN   NaN    NaN  325.029999  326.950012  315.010010   
2011-11-17  NaN  NaN NaN   NaN    NaN  316.709991  319.640015  310.000000   
2011-11-18  NaN  NaN NaN   NaN    NaN  314.299988  315.899994  308.260010   
...         ...  ...  ..   ...    ...         ...         ...         ...   
2012-04-02  NaN  NaN NaN   NaN    NaN  417.000000  421.410004  413.540009   
2012-04-03  NaN  NaN NaN   NaN    NaN  417.579987  422.429993  417.579987   
2012-04-04  NaN  NaN NaN   NaN    NaN  419.230011  420.910004  411.119995   
2012-04-05  NaN  NaN NaN   NaN    NaN  415.089996  424.989990  414.970001   
2012-04-09  NaN  NaN NaN   NaN    NaN  420.500000  427.130005  420.500000   

                                ...                                          \
                 Close  Volume  ...      NATR     SMA_5    SMA_20   SMA_100   
Date                            ...                                           
2011-11-14  327.029999  464600  ...       NaN       NaN       NaN       NaN   
2011-11-15  327.459991  496600  ...       NaN       NaN       NaN       NaN   
2011-11-16  316.670013  791900  ...       NaN       NaN       NaN       NaN   
2011-11-17  313.609985  710200  ...       NaN       NaN       NaN       NaN   
2011-11-18  311.059998  506100  ...       NaN  1.026059       NaN       NaN   
...                ...     ...  ...       ...       ...       ...       ...   
2012-04-02  418.399994  381300  ...  1.488409  1.003098  0.978846       NaN   
2012-04-03  422.260010  482800  ...  1.451501  0.994349  0.974037       NaN   
2012-04-04  417.040009  678900  ...  1.555494  1.005333  0.989219       NaN   
2012-04-05  424.980011  539400  ...  1.585812  0.988602  0.973891       NaN   
2012-04-09  426.309998  461700  ...  1.579032  0.989416  0.974158  0.855532   

                                                                       
                 SAR Williams%R       APO     ROC_5    ROC_20 ROC_100  
Date                                                                   
2011-11-14       NaN        NaN       NaN       NaN       NaN     NaN  
2011-11-15  1.021010        NaN       NaN       NaN       NaN     NaN  
2011-11-16  1.055799        NaN       NaN       NaN       NaN     NaN  
2011-11-17  1.066101        NaN       NaN       NaN       NaN     NaN  
2011-11-18  1.074841        NaN       NaN       NaN       NaN     NaN  
...              ...        ...       ...       ...       ...     ...  
2012-04-02  0.717017 -28.250379  0.028758 -0.014834  0.064685     NaN  
2012-04-03  0.710463 -15.974781  0.027696  0.002112  0.090266     NaN  
2012-04-04  0.719355 -45.294653  0.026353 -0.007260  0.063606     NaN  
2012-04-05  0.705916  -8.187411  0.025089  0.010366  0.067306     NaN  
2012-04-09  0.703713  -4.591302  0.024099  0.019880  0.070808     NaN  

[100 rows x 380 columns]

At this point, we have a 3D nested dataframe, split by tickers and then by columns. To put this data in CSV files, we will create a CSV file for each ticker containing the relevant information.

In [9]:
import os

for ticker in tickers:
    print(f"Saving data for {ticker}")
    
    tickerdata = data[ticker].dropna() #remove nan values
    if not os.path.exists("data"):
        os.mkdir("data")
    tickerdata.to_csv(f'./data/{ticker}.csv') #write to csv file

print("======================")
print("Saved stocks to CSV files. Take a look inside /data for them.")

Saving data for AMZN
Saving data for TSLA
Saving data for BABA
Saving data for HD
Saving data for TM
Saving data for NKE
Saving data for MCD
Saving data for LOW
Saving data for SBUX
Saving data for JD
Saving data for PDD
Saving data for BKNG
Saving data for GM
Saving data for MELI
Saving data for TJX
Saving data for NIO
Saving data for F
Saving data for LULU
Saving data for HMC
Saving data for CMG
Saved stocks to CSV files. Take a look inside /data for them.
